# Data Understanding & Preparation

In [1]:
# ============================================================
# STAGE 1 — Data Understanding & Preparation (STRATEGIC REVISION)
# Added:
# - Pothole area ratio analysis
# - Small vs large pothole distribution
# - Data bias diagnosis (ALL-positive training set)
# - Early feasibility signal for Dice ~0.90
# ============================================================

from pathlib import Path
import numpy as np
import cv2
import pandas as pd
from tqdm import tqdm
import re

# -----------------------------
# CONFIG
# -----------------------------
DATA_ROOT = Path("/kaggle/input/data-science-ara-7-0/dataset/dataset")
TRAIN_IMG_DIR = DATA_ROOT / "train" / "images"
TRAIN_MASK_DIR = DATA_ROOT / "train" / "mask"
TEST_IMG_DIR  = DATA_ROOT / "test" / "images"
IMG_EXTS = {".jpg", ".jpeg", ".png"}

# -----------------------------
# 1. LOAD FILES
# -----------------------------
train_images = sorted([p for p in TRAIN_IMG_DIR.iterdir() if p.suffix.lower() in IMG_EXTS])
train_masks  = sorted([p for p in TRAIN_MASK_DIR.iterdir() if p.suffix.lower() in IMG_EXTS])
test_images  = sorted([p for p in TEST_IMG_DIR.iterdir() if p.suffix.lower() in IMG_EXTS])

print(f"[INFO] Train images : {len(train_images)}")
print(f"[INFO] Train masks  : {len(train_masks)}")
print(f"[INFO] Test images  : {len(test_images)}")

# -----------------------------
# 2. BUILD MASK INDEX
# -----------------------------
def extract_index(name: str):
    m = re.search(r"(\d+)", name)
    return m.group(1) if m else None

mask_index = {}
for m in train_masks:
    idx = extract_index(m.stem)
    if idx is not None:
        mask_index[idx] = m

# -----------------------------
# 3. PAIR IMAGE–MASK
# -----------------------------
pairs = []
for img in train_images:
    idx = extract_index(img.stem)
    if idx in mask_index:
        pairs.append({
            "image_path": img,
            "mask_path": mask_index[idx],
            "id": idx
        })

assert len(pairs) > 0, "No valid image-mask pairs found"
print(f"[INFO] Valid image-mask pairs: {len(pairs)}")

# -----------------------------
# 4. SANITY + STRUCTURAL ANALYSIS
# -----------------------------
records = []

for p in tqdm(pairs, desc="Analyzing dataset"):
    img = cv2.imread(str(p["image_path"]))
    mask = cv2.imread(str(p["mask_path"]), cv2.IMREAD_GRAYSCALE)

    assert img is not None
    assert mask is not None
    assert img.shape[:2] == mask.shape

    h, w = mask.shape
    total_pixels = h * w
    pothole_pixels = (mask == 255).sum()
    area_ratio = pothole_pixels / total_pixels

    records.append({
        "image": p["image_path"].name,
        "height": h,
        "width": w,
        "pothole_pixels": pothole_pixels,
        "area_ratio": area_ratio,
        "has_pothole": int(pothole_pixels > 0)
    })

df = pd.DataFrame(records)

# -----------------------------
# 5. CRITICAL DATASET INSIGHTS
# -----------------------------
print("\n[INSIGHT] Pothole presence:")
print(df["has_pothole"].value_counts())

print("\n[INSIGHT] Pothole area ratio summary:")
print(df["area_ratio"].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]))

print("\n[INSIGHT] Images by pothole size category:")
print(pd.cut(
    df["area_ratio"],
    bins=[0, 0.005, 0.02, 0.05, 1.0],
    labels=["very_small", "small", "medium", "large"]
).value_counts())

# -----------------------------
# 6. DATA-DRIVEN FEASIBILITY SIGNAL
# -----------------------------
small_ratio = (df["area_ratio"] < 0.01).mean()

print("\n[FEASIBILITY CHECK]")
print(f"Images with pothole <1% area: {small_ratio:.2%}")

if small_ratio > 0.5:
    print("WARNING: Majority potholes are VERY SMALL")
    print("Dice ~0.90 will be EXTREMELY difficult due to boundary sensitivity")
else:
    print("Pothole size distribution is favorable for higher Dice")

# -----------------------------
# 7. FINAL MANIFEST
# -----------------------------
df_manifest = pd.DataFrame({
    "image_path": [str(p["image_path"]) for p in pairs],
    "mask_path":  [str(p["mask_path"]) for p in pairs],
    "id":         [p["id"] for p in pairs],
})

print(f"\n[INFO] Final training samples: {len(df_manifest)}")

print("\n[STAGE 1 COMPLETE — STRATEGIC]")
print("Dataset validated")
print("Structural bias diagnosed")
print("Upper-bound Dice feasibility assessed")
print("Ready to proceed with informed strategy to STAGE 2")

[INFO] Train images : 498
[INFO] Train masks  : 498
[INFO] Test images  : 295
[INFO] Valid image-mask pairs: 498


Analyzing dataset: 100%|██████████| 498/498 [00:25<00:00, 19.56it/s]


[INSIGHT] Pothole presence:
has_pothole
1    498
Name: count, dtype: int64

[INSIGHT] Pothole area ratio summary:
count    498.000000
mean       0.134860
std        0.128772
min        0.000235
10%        0.007938
25%        0.040943
50%        0.091678
75%        0.193834
90%        0.329536
max        0.674005
Name: area_ratio, dtype: float64

[INSIGHT] Images by pothole size category:
area_ratio
large         344
medium         80
very_small     43
small          31
Name: count, dtype: int64

[FEASIBILITY CHECK]
Images with pothole <1% area: 11.85%
Pothole size distribution is favorable for higher Dice

[INFO] Final training samples: 498

[STAGE 1 COMPLETE — STRATEGIC]
Dataset validated
Structural bias diagnosed
Upper-bound Dice feasibility assessed
Ready to proceed with informed strategy to STAGE 2


# Preprocessing & Data Augmentation

In [2]:
# ============================================================
# STAGE 2 — Preprocessing & Data Augmentation (FINAL PIPELINE)
# Goal:
# - Boundary-safe augmentation
# - Multi-scale ready (512 & 640)
# - Fully compatible with STAGE 3 top-score training
# ============================================================

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# -----------------------------
# GLOBAL NORMALIZATION
# -----------------------------
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)

# ============================================================
# 1. TRAIN AUGMENTATION — 512 (BASE CONTEXT)
# ============================================================
train_transform_512 = A.Compose(
    [
        A.Resize(512, 512, interpolation=cv2.INTER_LINEAR),
        A.HorizontalFlip(p=0.5),

        A.Affine(
            scale=(0.95, 1.10),
            translate_percent=(0.0, 0.05),
            rotate=(-3, 3),
            interpolation=cv2.INTER_LINEAR,
            mode=cv2.BORDER_REFLECT_101,
            p=0.5
        ),

        A.RandomBrightnessContrast(
            brightness_limit=0.25,
            contrast_limit=0.25,
            p=0.7
        ),

        A.HueSaturationValue(
            hue_shift_limit=8,
            sat_shift_limit=15,
            val_shift_limit=8,
            p=0.4
        ),

        A.RandomShadow(
            shadow_roi=(0, 0.4, 1, 1),
            p=0.3
        ),

        A.OneOf(
            [
                A.GaussianBlur(blur_limit=3),
                A.GaussNoise(),
            ],
            p=0.2
        ),

        A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
        ToTensorV2(),
    ],
    additional_targets={"mask": "mask"},
)

# ============================================================
# 2. TRAIN AUGMENTATION — 640 (BOUNDARY & DETAIL)
# ============================================================
train_transform_640 = A.Compose(
    [
        A.Resize(640, 640, interpolation=cv2.INTER_LINEAR),
        A.HorizontalFlip(p=0.5),

        A.Affine(
            scale=(0.98, 1.05),          # lebih konservatif
            translate_percent=(0.0, 0.03),
            rotate=(-2, 2),
            interpolation=cv2.INTER_LINEAR,
            mode=cv2.BORDER_REFLECT_101,
            p=0.4
        ),

        A.RandomBrightnessContrast(
            brightness_limit=0.20,
            contrast_limit=0.20,
            p=0.6
        ),

        A.HueSaturationValue(
            hue_shift_limit=6,
            sat_shift_limit=12,
            val_shift_limit=6,
            p=0.3
        ),

        A.RandomShadow(
            shadow_roi=(0, 0.4, 1, 1),
            p=0.25
        ),

        A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
        ToTensorV2(),
    ],
    additional_targets={"mask": "mask"},
)

# ============================================================
# 3. VALIDATION AUGMENTATION (DETERMINISTIC)
# ============================================================
valid_transform = A.Compose(
    [
        A.Resize(512, 512, interpolation=cv2.INTER_LINEAR),
        A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
        ToTensorV2(),
    ],
    additional_targets={"mask": "mask"},
)

# ============================================================
# 4. TEST AUGMENTATION (MATCH VALIDATION)
# ============================================================
test_transform = A.Compose(
    [
        A.Resize(512, 512, interpolation=cv2.INTER_LINEAR),
        A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
        ToTensorV2(),
    ]
)

# ============================================================
# 5. SUMMARY
# ============================================================
print("[STAGE 2 — FINAL PIPELINE READY]")
print("✔ Boundary-safe augmentations")
print("✔ Multi-scale training support (512 & 640)")
print("✔ Deterministic validation & test")
print("✔ Fully compatible with STAGE 3 top-score")

[STAGE 2 — FINAL PIPELINE READY]
✔ Boundary-safe augmentations
✔ Multi-scale training support (512 & 640)
✔ Deterministic validation & test
✔ Fully compatible with STAGE 3 top-score


/tmp/ipykernel_24/1110423978.py:27: UserWarning: Argument(s) 'mode' are not valid for transform Affine
  A.Affine(
/tmp/ipykernel_24/1110423978.py:76: UserWarning: Argument(s) 'mode' are not valid for transform Affine
  A.Affine(


# Model Construction & Training

In [3]:
!pip install -q segmentation-models-pytorch==0.3.3 timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 52.4 MB/s eta 0:00:00


In [4]:
# ============================================================
# STAGE 3 — Model Construction & Training (TOP SCORE, ALIGNED)
# Key upgrades:
# - Multi-scale curriculum (512 → 640)
# - Boundary-aware loss
# - Fully aligned with STAGE 2 FINAL
# ============================================================

import os, re, random
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR

import segmentation_models_pytorch as smp

# -----------------------------
# REPRODUCIBILITY
# -----------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# -----------------------------
# DATA
# -----------------------------
DATA_ROOT = Path("/kaggle/input/data-science-ara-7-0/dataset/dataset")
TRAIN_IMG_DIR = DATA_ROOT / "train" / "images"
TRAIN_MASK_DIR = DATA_ROOT / "train" / "mask"

def extract_idx(name):
    return re.search(r"(\d+)", name).group(1)

pairs = []
for img in TRAIN_IMG_DIR.iterdir():
    idx = extract_idx(img.name)
    mask = TRAIN_MASK_DIR / f"mask_{idx}.png"
    if mask.exists():
        pairs.append((str(img), str(mask)))

df = pd.DataFrame(pairs, columns=["image_path", "mask_path"])

from sklearn.model_selection import train_test_split
df_train, df_valid = train_test_split(df, test_size=0.15, random_state=SEED)

# -----------------------------
# DATASET
# -----------------------------
class PotholeDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img = cv2.imread(self.df.loc[idx, "image_path"])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        mask = cv2.imread(self.df.loc[idx, "mask_path"], cv2.IMREAD_GRAYSCALE)
        mask = (mask == 255).astype("float32")

        aug = self.transform(image=img, mask=mask)
        img = aug["image"]
        mask = aug["mask"].unsqueeze(0)
        return img, mask

# -----------------------------
# VALIDATION LOADER (FIXED)
# -----------------------------
valid_loader = DataLoader(
    PotholeDataset(df_valid, valid_transform),
    batch_size=4,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

# -----------------------------
# MULTI-SCALE CURRICULUM
# -----------------------------
def get_train_transform(epoch):
    # first half: context & stability
    if epoch < 15:
        return train_transform_512
    # second half: boundary refinement
    return train_transform_640

# -----------------------------
# MODEL
# -----------------------------
model = smp.UnetPlusPlus(
    encoder_name="efficientnet-b4",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
).to(device)

# -----------------------------
# LOSS (BOUNDARY-AWARE)
# -----------------------------
dice_loss = smp.losses.DiceLoss(mode="binary", from_logits=True)
focal_loss = smp.losses.FocalLoss(mode="binary", gamma=2.0)

def boundary_loss(prob, target):
    sobel = torch.tensor(
        [[1,0,-1],[2,0,-2],[1,0,-1]],
        device=prob.device
    ).float().view(1,1,3,3)

    prob_edge = torch.abs(
        nn.functional.conv2d(prob, sobel, padding=1)
    )
    target_edge = torch.abs(
        nn.functional.conv2d(target, sobel, padding=1)
    )

    return nn.functional.l1_loss(prob_edge, target_edge)

def criterion(logits, target):
    prob = torch.sigmoid(logits).clamp(1e-4, 1 - 1e-4)
    return (
        dice_loss(logits, target)
        + 0.5 * focal_loss(logits, target)
        + 0.3 * boundary_loss(prob, target)
    )

# -----------------------------
# OPTIMIZER
# -----------------------------
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=30)

# -----------------------------
# METRIC
# -----------------------------
def dice_coef(prob, target, eps=1e-7):
    pred = (prob > 0.5).float()
    intersection = (pred * target).sum(dim=(2,3))
    union = pred.sum(dim=(2,3)) + target.sum(dim=(2,3))
    return ((2 * intersection + eps) / (union + eps)).mean()

# -----------------------------
# TRAINING LOOP
# -----------------------------
EPOCHS = 30
best_dice = 0.0

for epoch in range(EPOCHS):
    model.train()

    train_loader = DataLoader(
        PotholeDataset(df_train, get_train_transform(epoch)),
        batch_size=4,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )

    for imgs, masks in tqdm(train_loader, desc=f"Epoch {epoch+1} [TRAIN]"):
        imgs, masks = imgs.to(device), masks.to(device)

        optimizer.zero_grad()
        logits = model(imgs)
        loss = criterion(logits, masks)
        loss.backward()
        optimizer.step()

    scheduler.step()

    model.eval()
    val_dice = 0.0
    with torch.no_grad():
        for imgs, masks in valid_loader:
            imgs, masks = imgs.to(device), masks.to(device)
            prob = torch.sigmoid(model(imgs))
            val_dice += dice_coef(prob, masks).item()

    val_dice /= len(valid_loader)

    print(f"Epoch {epoch+1:02d} | ValDice {val_dice:.4f}")

    if val_dice > best_dice:
        best_dice = val_dice
        torch.save(model.state_dict(), "/kaggle/working/best_unetpp_effb4.pt")
        print(">> Best model saved")

print("\n[STAGE 3 COMPLETE — TOP SCORE ALIGNED]")
print("Best Validation Dice:", round(best_dice, 4))
print("READY FOR STAGE 4")

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Device: cuda
Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b4-6ed6700e.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b4-6ed6700e.pth


100%|██████████| 74.4M/74.4M [00:00<00:00, 204MB/s]
Epoch 1 [TRAIN]: 100%|██████████| 106/106 [00:42<00:00,  2.51it/s]


Epoch 01 | ValDice 0.5689
>> Best model saved


Epoch 2 [TRAIN]: 100%|██████████| 106/106 [00:40<00:00,  2.59it/s]


...

Epoch 28 | ValDice 0.7238


Epoch 29 [TRAIN]: 100%|██████████| 106/106 [01:01<00:00,  1.72it/s]


Epoch 29 | ValDice 0.7246


Epoch 30 [TRAIN]: 100%|██████████| 106/106 [01:01<00:00,  1.72it/s]


Epoch 30 | ValDice 0.7255

[STAGE 3 COMPLETE — TOP SCORE ALIGNED]
Best Validation Dice: 0.7297
READY FOR STAGE 4


# Optimization, Validation & Refinement

In [5]:
# ============================================================
# STAGE 4 — Optimization, Validation & Refinement (OPTUNA)
# Goal:
# - Optimize decision-level parameters (threshold + post-process)
# - Maximize validation Dice
# ============================================================

# -----------------------------
# 0. INSTALL OPTUNA
# -----------------------------
!pip install -q optuna

# -----------------------------
# 1. IMPORTS
# -----------------------------
import optuna
import numpy as np
import torch
import cv2
from tqdm import tqdm
import pandas as pd

# -----------------------------
# 2. CONFIG
# -----------------------------
MODEL_PATH = "/kaggle/working/best_unetpp_effb4.pt"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

N_TRIALS = 40   # cukup untuk lonjakan besar, masih aman runtime

# -----------------------------
# 3. LOAD MODEL
# -----------------------------
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()
model.to(DEVICE)

print("[INFO] Best model loaded for Optuna")

# -----------------------------
# 4. HELPER FUNCTIONS
# -----------------------------
def dice_score(pred, target, eps=1e-7):
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum()
    return (2 * intersection + eps) / (union + eps)

def remove_small_objects(mask, min_area):
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(
        mask.astype(np.uint8), connectivity=8
    )
    clean = np.zeros_like(mask, dtype=np.uint8)
    for i in range(1, num_labels):
        if stats[i, cv2.CC_STAT_AREA] >= min_area:
            clean[labels == i] = 1
    return clean

# -----------------------------
# 5. OPTUNA OBJECTIVE
# -----------------------------
def objective(trial):
    threshold = trial.suggest_float("threshold", 0.25, 0.55)
    min_area  = trial.suggest_int("min_area", 100, 1500, step=100)

    dices = []

    with torch.no_grad():
        for imgs, masks in valid_loader:
            imgs = imgs.to(DEVICE)
            masks = masks.to(DEVICE)

            probs = torch.sigmoid(model(imgs)).cpu().numpy()
            gt = masks.cpu().numpy()

            for i in range(probs.shape[0]):
                pred = (probs[i, 0] > threshold).astype(np.uint8)
                pred = remove_small_objects(pred, min_area)

                dice = dice_score(pred, gt[i, 0])
                dices.append(dice)

    return float(np.mean(dices))

# -----------------------------
# 6. RUN OPTUNA
# -----------------------------
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=N_TRIALS)

# -----------------------------
# 7. BEST CONFIG
# -----------------------------
best_params = study.best_params
best_dice = study.best_value

print("\n[OPTUNA BEST CONFIG]")
print(best_params)
print(f"Best validation Dice: {best_dice:.4f}")

# -----------------------------
# 8. SAVE CONFIG FOR STAGE 5
# -----------------------------
OPT_CONFIG = {
    "threshold": best_params["threshold"],
    "min_area": best_params["min_area"],
}

print("\n[STAGE 4 COMPLETE — OPTUNA STRATEGIC]")
print("Decision-level optimization finished")
print("Ready for STAGE 5 (Inference & Submission)")

[I 2026-02-05 04:35:46,978] A new study created in memory with name: no-name-0b906a62-3736-4a8e-870b-c3157f29c981


[INFO] Best model loaded for Optuna


[I 2026-02-05 04:35:49,821] Trial 0 finished with value: 0.6950937324368889 and parameters: {'threshold': 0.3760305407879275, 'min_area': 700}. Best is trial 0 with value: 0.6950937324368889.
[I 2026-02-05 04:35:52,600] Trial 1 finished with value: 0.6919575880308166 and parameters: {'threshold': 0.46298269680077875, 'min_area': 1100}. Best is trial 0 with value: 0.6950937324368889.
[I 2026-02-05 04:35:55,367] Trial 2 finished with value: 0.7037913534620266 and parameters: {'threshold': 0.3915228893015128, 'min_area': 300}. Best is trial 2 with value: 0.7037913534620266.
[I 2026-02-05 04:35:58,088] Trial 3 finished with value: 0.695539153554166 and parameters: {'threshold': 0.335954758620002, 'min_area': 500}. Best is trial 2 with value: 0.7037913534620266.
[I 2026-02-05 04:36:00,806] Trial 4 finished with value: 0.6933610231767795 and parameters: {'threshold': 0.5389112626368613, 'min_area': 600}. Best is trial 2 with value: 0.7037913534620266.
[I 2026-02-05 04:36:03,534] Trial 5 fini


[OPTUNA BEST CONFIG]
{'threshold': 0.2517684134212907, 'min_area': 100}
Best validation Dice: 0.7395

[STAGE 4 COMPLETE — OPTUNA STRATEGIC]
Decision-level optimization finished
Ready for STAGE 5 (Inference & Submission)


# Inference, Encoding & Submission

In [6]:
# ============================================================
# STAGE 5 — Inference, RLE Encoding & Submission (STRATEGIC FINAL)
# Goal:
# - Stabilize predictions (TTA)
# - Use Optuna-optimized decision parameters
# - Refine boundary for Dice maximization
# ============================================================

import numpy as np
import pandas as pd
import torch
import cv2
from pathlib import Path
from tqdm import tqdm

# -----------------------------
# CONFIG
# -----------------------------
DATA_ROOT = Path("/kaggle/input/data-science-ara-7-0/dataset/dataset")
TEST_IMG_DIR = DATA_ROOT / "test" / "images"
SAMPLE_SUB = Path("/kaggle/input/data-science-ara-7-0/sample_submission.csv")

MODEL_PATH = "/kaggle/working/best_unetpp_effb4.pt"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BEST_THRESHOLD = OPT_CONFIG["threshold"]
MIN_AREA = OPT_CONFIG["min_area"]
INPUT_SIZE = 512

# -----------------------------
# LOAD MODEL
# -----------------------------
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()
model.to(DEVICE)

# -----------------------------
# RLE ENCODER (OFFICIAL)
# -----------------------------
def encode_rle(mask: np.ndarray, pos_value: int = 255) -> str:
    binary = (mask == pos_value).astype(np.uint8)
    pixels = binary.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[0::2]
    return " ".join(str(x) for x in runs)

# -----------------------------
# POST-PROCESSING
# -----------------------------
def remove_small_objects(mask, min_area):
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(
        mask.astype(np.uint8), connectivity=8
    )
    clean = np.zeros_like(mask, dtype=np.uint8)
    for i in range(1, num_labels):
        if stats[i, cv2.CC_STAT_AREA] >= min_area:
            clean[labels == i] = 1
    return clean

def fill_holes(mask):
    return cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8))

# -----------------------------
# COLLECT TEST FILES
# -----------------------------
test_images = sorted(TEST_IMG_DIR.glob("*.jpg"))
print("[INFO] Test images:", len(test_images))

# -----------------------------
# INFERENCE WITH LIGHT TTA
# -----------------------------
records = []

with torch.no_grad():
    for img_path in tqdm(test_images, desc="Inference + TTA"):
        img_name = img_path.name

        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h0, w0 = img.shape[:2]

        img_resized = cv2.resize(img, (INPUT_SIZE, INPUT_SIZE))
        img_resized = img_resized.astype("float32") / 255.0

        img_resized[..., 0] = (img_resized[..., 0] - 0.485) / 0.229
        img_resized[..., 1] = (img_resized[..., 1] - 0.456) / 0.224
        img_resized[..., 2] = (img_resized[..., 2] - 0.406) / 0.225

        # original
        x = torch.from_numpy(img_resized.transpose(2,0,1)).unsqueeze(0).to(DEVICE)
        prob1 = torch.sigmoid(model(x))[0,0]

        # horizontal flip TTA
        x_flip = torch.flip(x, dims=[3])
        prob2 = torch.sigmoid(model(x_flip))[0,0]
        prob2 = torch.flip(prob2, dims=[1])

        # average probability
        prob = ((prob1 + prob2) / 2).cpu().numpy()

        # threshold + refinement
        pred = (prob > BEST_THRESHOLD).astype(np.uint8)
        pred = remove_small_objects(pred, MIN_AREA)
        pred = fill_holes(pred)

        pred = cv2.resize(pred, (w0, h0), interpolation=cv2.INTER_NEAREST)
        pred_255 = pred * 255

        rle = "" if pred_255.sum() == 0 else encode_rle(pred_255)

        records.append({
            "ImageId": img_name,
            "rle": rle
        })

# -----------------------------
# BUILD SUBMISSION
# -----------------------------
df_sub = pd.DataFrame(records)
df_sub = df_sub[pd.read_csv(SAMPLE_SUB).columns.tolist()]

OUT_SUB = "/kaggle/working/submission.csv"
df_sub.to_csv(OUT_SUB, index=False)

print("\n[STAGE 5 COMPLETE — STRATEGIC FINAL]")
print("Submission saved to:", OUT_SUB)
print("Rows:", len(df_sub))
print("Empty RLE:", (df_sub["rle"] == "").sum())
print(df_sub.head())

[INFO] Test images: 295


Inference + TTA: 100%|██████████| 295/295 [00:26<00:00, 11.29it/s]



[STAGE 5 COMPLETE — STRATEGIC FINAL]
Submission saved to: /kaggle/working/submission.csv
Rows: 295
Empty RLE: 0
        ImageId                                                rle
0  test_001.jpg  5242 3 5540 6 5840 7 6139 8 6439 8 6738 10 703...
1  test_002.jpg  123508 3 124227 4 124947 5 125667 5 126387 5 1...
2  test_003.jpg  1338525 4 1340821 4 1343117 4 1345413 4 134770...
3  test_004.jpg  17820 3 18120 5 18420 5 18719 6 19019 7 19319 ...
4  test_005.jpg  50529 6 50828 8 51126 12 51426 12 51726 14 517...
